In [ ]:
print("Activating ipynb")
print ("Importing libs")
import os 
import numpy as np
import pandas as pd



In [ ]:
# Load Initial Data
print("Loading Initial Game Data")
games_folder_path = "C:/Users/Craig/Documents/Thesis/Thomas_Gallagher_Thesis/Data/AFL_Stats_sorted/Year/Games/"
games_raw_2012 = pd.read_csv(games_folder_path + '2012.csv', index_col=False)
games_raw_2013 = pd.read_csv(games_folder_path + '2013.csv', index_col=False)
games_raw_2014 = pd.read_csv(games_folder_path + '2014.csv', index_col=False)
games_raw_2015 = pd.read_csv(games_folder_path + '2015.csv', index_col=False)
games_raw_2016 = pd.read_csv(games_folder_path + '2016.csv', index_col=False)
games_raw_2017 = pd.read_csv(games_folder_path + '2017.csv', index_col=False)
games_raw_2018 = pd.read_csv(games_folder_path + '2018.csv', index_col=False)
games_raw_2019 = pd.read_csv(games_folder_path + '2019.csv', index_col=False)
games_raw_2020 = pd.read_csv(games_folder_path + '2020.csv', index_col=False)
games_raw_2021 = pd.read_csv(games_folder_path + '2021.csv', index_col=False)

all_games = os.path.join("C:/Users/Craig/Documents/Thesis/Thomas_Gallagher_Thesis/Data/AFL_Stats_sorted/games_sorted.csv")
all_games_raw = pd.read_csv(all_games, index_col=False)

print(all_games)

In [ ]:
# Load Stats
stats_folder_path = "C:/Users/Craig/Documents/Thesis/Thomas_Gallagher_Thesis/Data/AFL_Stats_sorted/Year/Players/"
stats_raw_2012 = pd.read_csv(stats_folder_path + '2012.csv', index_col=False)
stats_raw_2013 = pd.read_csv(stats_folder_path + '2013.csv', index_col=False)
stats_raw_2014 = pd.read_csv(stats_folder_path + '2014.csv', index_col=False)
stats_raw_2015 = pd.read_csv(stats_folder_path + '2015.csv', index_col=False)
stats_raw_2016 = pd.read_csv(stats_folder_path + '2016.csv', index_col=False)
stats_raw_2017 = pd.read_csv(stats_folder_path + '2017.csv', index_col=False)
stats_raw_2018 = pd.read_csv(stats_folder_path + '2018.csv', index_col=False)
stats_raw_2019 = pd.read_csv(stats_folder_path + '2019.csv', index_col=False)
stats_raw_2020 = pd.read_csv(stats_folder_path + '2020.csv', index_col=False)
stats_raw_2021 = pd.read_csv(stats_folder_path + '2021.csv', index_col=False)

all_stats = "C:/Users/Craig/Documents/Thesis/Thomas_Gallagher_Thesis/Data/AFL_Stats_sorted/stats_sorted.csv"
all_stats_raw = pd.read_csv(all_stats, index_col=False)